In [3]:
!pip install tweepy

     |████████████████████████████████| 147 kB 978 kB/s eta 0:00:01


In [5]:
!pip install textblob

     |████████████████████████████████| 636 kB 546 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 190 kB/s eta 0:00:01
     |████████████████████████████████| 730 kB 107 kB/s eta 0:00:01
     |████████████████████████████████| 303 kB 168 kB/s eta 0:00:01


In [17]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

'''
Generic Twitter Class for sentiment analysis.
'''
    
def TwitterClient():
    '''
    Class constructor or initialization method.
    '''
    # keys and tokens from the Twitter Dev Console
    consumer_key = ''
    consumer_secret = ''
    access_token = ''
    access_token_secret = ''

    # attempt authentication
    try:
        
        # create OAuthHandler object
        auth = OAuthHandler(consumer_key, consumer_secret)
        
        # set access token and secret
        auth.set_access_token(access_token, access_token_secret)
        
        # create tweepy API object to fetch tweets
        api = tweepy.API(auth)
        
        return api
        
    except:
        print("Error: Authentication Failed")
        
        

def clean_tweet(tweet):
    
    '''
    Utility function to clean tweet text by removing links, special characters
    using simple regex statements.
    '''
    
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())



def get_tweet_sentiment(tweet):
    
    '''
    Utility function to classify sentiment of passed tweet
    using textblob's sentiment method
    '''
    
    # create TextBlob object of passed tweet text
    analysis = TextBlob(clean_tweet(tweet))
    
    # set sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    
    else:
        return 'negative'
    
    

def get_tweets(api, query, count = 10):
    
    '''
    Main function to fetch tweets and parse them.
    '''
    # empty list to store parsed tweets
    tweets = []

    try:
        # call twitter api to fetch tweets
        fetched_tweets = api.search(q = query, count = count)

        # parsing tweets one by one
        for tweet in fetched_tweets:
            
            # empty dictionary to store required params of a tweet
            parsed_tweet = {}

            # saving text of tweet
            parsed_tweet['text'] = tweet.text
            
            # saving sentiment of tweet
            parsed_tweet['sentiment'] = get_tweet_sentiment(tweet.text)

            # appending parsed tweet to tweets list
            if tweet.retweet_count > 0:
                
                # if tweet has retweets, ensure that it is appended only once
                if parsed_tweet not in tweets:
                    tweets.append(parsed_tweet)
                    
            else:
                tweets.append(parsed_tweet)

        # return parsed tweets
        return tweets

    except tweepy.TweepError as e:
        # print error (if any)
        print("Error : " + str(e))


In [18]:
def main():
    
	# calling TwitterClient Function
	api = TwitterClient()
    
	# calling function to get tweets
	tweets = get_tweets(api, query = '#Donald Trump', count = 50)

	# picking positive tweets from tweets
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    
	# percentage of positive tweets
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    
	# picking negative tweets from tweets
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    
	# percentage of negative tweets
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    
	# percentage of neutral tweets
	print("Neutral tweets percentage: {} % \
		".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))

	# printing first 5 positive tweets
	print("\n\nPositive tweets:")
	for tweet in ptweets[:3]:
		print(tweet['text'])
        

	# printing first 5 negative tweets
	print("\n\nNegative tweets:")
	for tweet in ntweets[:10]:
		print(tweet['text'])
        
        

if __name__ == "__main__":
	# calling main function
	main()

Positive tweets percentage: 24.0 %
Negative tweets percentage: 8.0 %
Neutral tweets percentage: 68.0 % 		


Positive tweets:
#SNL 1st #HINT #Trump was right again.  the Government always saves / preserves / secures itself first.  As the… https://t.co/27FJC7VrTP
Sound clip: Boy is she smart https://t.co/zphcpNMdfp #donaldtrump #donald #trump #soundboard #boy #is #she #smart
RT @TruNews: 📣 New Podcast! "Fauci’s Friend Used NIH Grant to Weaponize COVID in Wuhan" on @Spreaker #biden #christ #christian #day #donald…


Negative tweets:
RT @breaknnews1: Lawyer files assault complaint vs. Rams' Donald
#breakingnews #news #china #trump #biden #photo
#aarondonald #assault #com…
Lawyer files assault complaint vs. Rams' Donald
#breakingnews #news #china #trump #biden #photo
#aarondonald… https://t.co/qANAewizTT
@thehill Sad @SteveScalise didn’t follow up with the crisis at America's southern border which was exclusively crea… https://t.co/vDyTQjVk0Y
Man Accused Of Leaving Rep. #Madeleine #Dean Vul